In [ ]:
import subprocess

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
from matplotlib.pyplot import figure, show, subplots
from matplotlib import pyplot as plt

import numpy as np

In [ ]:
def ARCiS_initialize(atmosphere_key):
      splits = atmosphere_key.split("_")
      distance = int(splits[-1])
      
      config_ARCiS_loc = "config/config_ARCiS.in"
      save_location = f"/dataserver/users/formingworlds/borgmann/ARCiS_retrievals/{atmosphere_key}"
      atmosphere_file_loc = f"/Users/users/borgmann/Documents/masterproject/LIFEredoxsurvey/output/ARCiS/input_spectra/textfiles/ARCiS_input_{atmosphere_key}.txt"
      atmosphere_key_us = atmosphere_key.replace(".","_")
      command = f"ARCiS {config_ARCiS_loc} -o {save_location} -s obs1:file={atmosphere_file_loc} -s distance={distance}"
      prompt = (
        "ssh -n -T norma1;"
        "cd .. ;"
        f"tmux new-session -d -s '{atmosphere_key}'; "
        f"tmux send-keys -t {atmosphere_key_us}:0 'module unload ARCiS' C-m ;"
        f"tmux send-keys -t {atmosphere_key_us}:0 'module load ARCiS' C-m ;"
        f"tmux send-keys -t {atmosphere_key_us}:0 'module unload cfitsio' C-m ;"
        f"tmux send-keys -t {atmosphere_key_us}:0 'module load cfitsio' C-m ;"
        f"tmux send-keys -t {atmosphere_key_us}:0 'export OMP_NUM_THREADS=8' C-m ;"
        f"tmux send-keys -t {atmosphere_key_us}:0 '{command}' C-m"
        )
      subprocess.run(prompt, shell=True, check=True) 

In [ ]:
ARCiS_initialize("4.0_1.33_5")

In [ ]:
select_file = "-4.0_0.5_5"
retrieval_folder = "/dataserver/users/formingworlds/borgmann/ARCiS_retrievals/" + select_file
input_file = "../output/ARCiS/input_spectra/textfiles/ARCiS_input_" + select_file +".txt"
input_lambda = np.genfromtxt(input_file,usecols=0)
input_jansky = np.genfromtxt(input_file,usecols=1)

ARCiS_lambda = np.genfromtxt(retrieval_folder + "/emis_limits",usecols=0)
ARCiS_jansky = np.genfromtxt(retrieval_folder + "/emis_limits",usecols=4)
ARCiS_minsigma = np.genfromtxt(retrieval_folder + "/emis_limits",usecols=3)
ARCiS_plussigma = np.genfromtxt(retrieval_folder + "/emis_limits",usecols=5)
ARCiS_min3sigma = np.genfromtxt(retrieval_folder + "/emis_limits",usecols=1)
ARCiS_plus3sigma = np.genfromtxt(retrieval_folder + "/emis_limits",usecols=7)


fig = figure(figsize=(6,4.5))
frame1 = fig.add_subplot(1,1,1)
frame1.plot(input_lambda,input_jansky,color='blue', label='Input spectrum')
frame1.plot(ARCiS_lambda,ARCiS_jansky,color='black',label='Retrieved median',alpha=0.2)
frame1.fill_between(ARCiS_lambda,
                 ARCiS_minsigma,
                 ARCiS_plussigma,
                 color='black', alpha=0.2, label='1$\sigma$')
frame1.fill_between(ARCiS_lambda,
                 ARCiS_min3sigma,
                 ARCiS_plus3sigma,
                 color='black', alpha=0.1, label='3$\sigma$')

frame1.set_xlabel('wavelength (microns)')
frame1.set_ylabel('Janskys (10$^{-6}$)')
frame1.legend()

In [ ]:
retrieval_folder = "/dataserver/users/formingworlds/borgmann/ARCiS_retrievals/-1.33_0.5_5"
CO2_min133 = np.genfromtxt(retrieval_folder + "/pew_output.dat",usecols=3)
retrieval_folder = "/dataserver/users/formingworlds/borgmann/ARCiS_retrievals/-4.0_0.5_5"
CO2_min40 = np.genfromtxt(retrieval_folder + "/pew_output.dat",usecols=3)
retrieval_folder = "/dataserver/users/formingworlds/borgmann/ARCiS_retrievals/1.33_0.5_5"
CO2_133 = np.genfromtxt(retrieval_folder + "/pew_output.dat",usecols=3)
retrieval_folder = "/dataserver/users/formingworlds/borgmann/ARCiS_retrievals/4.0_0.5_5"
CO2_40 = np.genfromtxt(retrieval_folder + "/pew_output.dat",usecols=3)

fig = figure(figsize=(8,6))
frame1 = fig.add_subplot(1,1,1)
#frame1.step(range(len(CO2_min133)),CO2_min133)
frame1.hist(CO2_min133,bins=50,density=True,label='-1.33',alpha=0.5)
frame1.hist(CO2_min40,bins=50,density=True,label='-4.0',alpha=0.5)
#frame1.hist(CO2_133,bins=50,density=True,label='4.0',alpha=0.5)
frame1.set_xlabel('count')
frame1.set_ylabel('posterior')

frame1.legend()


# ARCiS_titles = ["H2O","CO2","CO","CH4","O2","SO2","N2","H2S","H2","S2"]
# print(ARCiS_titles)
# ARCiS_molecules = np.genfromtxt(retrieval_folder + "/mixingratios.dat",skip_header=1)[-1][2:]
# print(ARCiS_molecules)

In [ ]:
# one-time code, very messy
    #    H2O  2.2314E+01  8.9505E-01  2.4139E+02  (  2.4891E-03  9.4918E+02)
    #     H2  1.3636E-03  4.8136E-07  4.8637E-01  (  1.0276E-09  4.5167E+02)
    #    CO2  1.3297E-01  3.1349E-03  1.6111E+00  (  2.0191E-09  5.2100E+01)
    #     CO  7.7091E-04  9.3275E-07  3.5421E+00  (  1.8637E-09  7.5942E+02)
    #    SO2  6.5915E-07  1.4239E-08  4.0064E-05  (  1.3192E-09  2.1208E-03)
    #    H2S  2.5519E-04  2.0554E-07  1.7719E-01  (  1.9974E-09  4.2862E+01)
    #     S2  7.1882E-03  2.4950E-06  5.4561E+00  (  1.0680E-09  7.7595E+02)
    #     N2  1.2246E-03  2.2137E-07  5.0179E-01  (  1.5700E-09  8.9548E+02)
    #     O2  3.4343E-03  2.7499E-07  4.0267E+00  (  1.2394E-09  5.7755E+02)
    #    CH4  5.5202E-06  5.5046E-08  1.4066E-03  (  1.2811E-09  1.1600E+00)
    #     Rp  8.2679E-02  7.8766E-02  8.6787E-02  (  7.1545E-02  1.0022E-01)
    #     Mp  3.6415E-03  1.7981E-03  5.0982E-03  (  1.0916E-03  5.9937E-03)


names = ['H2O','H2','O2','CO2','CO','CH4','SO2','H2S','S2','N2']
volatile_colours = ["blue","red","gray","cyan","green","gray","orange","pink","purple","gray"]
true_fractions = [0.036179117384886224,0.677321414664592,4.895912375030067e-15,0.005783094467250954,0.2673756008646553,1.3589846090831264e-05,1.5468259619197373e-08,0.008735546796533558,9.403497825162163e-06,0.004582217009901435]
retrieved_fractions = [2.2314E+01,1.3636E-03,3.4343E-03,1.3297E-01,7.7091E-04,5.5202E-06,6.5915E-07,2.5519E-04,7.1882E-03,1.2246E-03]

# normalize
retrieved_sum = np.sum(retrieved_fractions)
retrieved_fractions_normalized = []
for i in retrieved_fractions:
  retrieved_fractions_normalized.append(i/retrieved_sum)

# start plot
mpl.rcParams['figure.dpi'] = 800
fig = figure(figsize=(8,6))
frame1 = fig.add_subplot(1,1,1)

# make the ratios cumulative, this is needed for the pie charts
true_cum = []
for i in range(len(true_fractions)):
    true_cum.append(sum(true_fractions[:i+1]))

# define the pie chart
circles = []
for i in range(len(true_cum)):
    if i == 0:
        x = [0] + np.cos(np.linspace(0, 2 * np.pi * true_cum[i], 100)).tolist()
        y = [0] + np.sin(np.linspace(0, 2 * np.pi * true_cum[i], 100)).tolist()
        xy = np.column_stack([x, y])
        circles.append(xy)
    else:
        x = [0] + np.cos(np.linspace(2 * np.pi * true_cum[i-1], 2 * np.pi * true_cum[i], 100)).tolist()
        y = [0] + np.sin(np.linspace(2 * np.pi * true_cum[i-1], 2 * np.pi * true_cum[i], 100)).tolist()
        xy = np.column_stack([x, y])
        circles.append(xy)

# plot
for i in range(len(true_cum)):
    frame1.scatter(0, 0, marker=circles[i], s=8000*(np.abs(circles[i]).max())**2, facecolor=volatile_colours[i])


# make the ratios cumulative, this is needed for the pie charts
true_cum = []
for i in range(len(retrieved_fractions_normalized)):
    true_cum.append(sum(retrieved_fractions_normalized[:i+1]))

# define the pie chart
circles = []
for i in range(len(true_cum)):
    if i == 0:
        x = [0] + np.cos(np.linspace(0, 2 * np.pi * true_cum[i], 100)).tolist()
        y = [0] + np.sin(np.linspace(0, 2 * np.pi * true_cum[i], 100)).tolist()
        xy = np.column_stack([x, y])
        circles.append(xy)
    else:
        x = [0] + np.cos(np.linspace(2 * np.pi * true_cum[i-1], 2 * np.pi * true_cum[i], 100)).tolist()
        y = [0] + np.sin(np.linspace(2 * np.pi * true_cum[i-1], 2 * np.pi * true_cum[i], 100)).tolist()
        xy = np.column_stack([x, y])
        circles.append(xy)

# plot
for i in range(len(true_cum)):
    frame1.scatter(0.2, 0, marker=circles[i], s=8000*(np.abs(circles[i]).max())**2, facecolor=volatile_colours[i])
frame1.set_xlim(-0.1,0.3)


In [ ]:

def savefigures(data,filename,title):
    volatile_labels = ['H2O', 'H2', 'O2', 'CO2', 'CO', 'CH4', 'SO2', 'H2S', 'S2', 'N2']
    volatile_colours = ["blue","red","gray","cyan","green","gray","orange","pink","purple","gray"]

    # start plot
    mpl.rcParams['figure.dpi'] = 800
    fig = figure(figsize=(8,6))
    frame1 = fig.add_subplot(1,1,1)

    for entry in range(len(data)):

        # find ratios of each molecule in the atmosphere
        ratios = []
        for i in range(len(data[entry])):
            ratios.append(data[entry][i]/sum(data[entry]))

        # make the ratios cumulative, this is needed for the pie charts
        ratios_cumulative = []
        for i in range(len(ratios)):
            ratios_cumulative.append(sum(ratios[:i+1]))

        # define the pie chart
        circles = []
        for i in range(len(ratios_cumulative)):
            if i == 0:
                x = [0] + np.cos(np.linspace(0, 2 * np.pi * ratios_cumulative[i], 100)).tolist()
                y = [0] + np.sin(np.linspace(0, 2 * np.pi * ratios_cumulative[i], 100)).tolist()
                xy = np.column_stack([x, y])
                circles.append(xy)
            else:
                x = [0] + np.cos(np.linspace(2 * np.pi * ratios_cumulative[i-1], 2 * np.pi * ratios_cumulative[i], 100)).tolist()
                y = [0] + np.sin(np.linspace(2 * np.pi * ratios_cumulative[i-1], 2 * np.pi * ratios_cumulative[i], 100)).tolist()
                xy = np.column_stack([x, y])
                circles.append(xy)

        # plot
        for i in range(len(ratios_cumulative)):
            frame1.scatter(coords[entry][0], coords[entry][1], marker=circles[i], s=800*(np.abs(circles[i]).max())**2, facecolor=volatile_colours[i])

    # make custom lines
    volatile_labels_custom = ['H2O', 'H2', 'CO2', 'CO', 'SO2', 'H2S', 'S2', 'N2', 'O2', 'CH4']
    volatile_colours_custom = ["blue","red","cyan","green","orange","pink","purple","gray","gray","gray"] # olive

    custom_lines = []
    for i in volatile_colours_custom:
        custom_lines.append(Line2D([0], [0], color=i, lw=4))

    frame1.legend(custom_lines,volatile_labels_custom,bbox_to_anchor=(1, 1.02))
    frame1.set_xlabel("Mantle $f$O$_2$, $\Delta$ IW")
    frame1.set_ylabel("Mantle H/C mass ratio")
    frame1.set_title(title)
    filepath = "../output/figures/" + filename
    plt.savefig(filepath)